# AVI Python SDK. CREATE, MODIFYING AND DELETING Objects

Having understood some basic concepts of how to use the sdk with the read operations, let's explore some operations that allow us to create, update and delete to have the complete picture.

https://avinetworks.com/docs/latest/api-guide/overview.html 
https://github.com/vmware/alb-sdk/tree/eng/python/avi/sdk 


- [1.- Initial Login](#1--initial-login)
- [2.- Create (POST) Operations](#2--create-post-method)
- [3.- Modify (PUT) Operations](#3--modify-put-method)
- [4.- Melete (DELETE) Operations](#3--modify-put-method)

## Initial Login

As a first step, we need to create the ApiSession object to interact with the controller via API. 

In [1]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:al7z8g6t34vzxc06vys7jvymg3igucoy


## 2.- CREATE (POST) Method

### Example 2.1: Sending manually created BODY

The POST method is used to create new objects. The way to invoke is quite similar to the GET method but this time we need to create a BODY containing information we want to update. Since error handling is important because we can make mistakes with fields names, it is crucial to add some logic to control AVI controller responses. In below example we will create a simple BODY manually including required fields to create a new vsvip object.

In [31]:
# Define POST parameters
url_path="vsvip"
vsvip_name = "vsvip-new-001"


body = {
  "vip": [
    {
      "enabled": "true",
      "auto_allocate_ip": "true",
      "auto_allocate_ip_type": "V4_ONLY",
      "ipam_network_subnet": {
          "network_ref": "https://192.168.1.15/api/network/network-5b30f803-879e-4312-b49b-309f41e98f7d",
          "subnet": {
            "ip_addr": {
               "addr": "192.168.1.0",
               "type": "V4"
            },
            "mask": 24
        }
      },
      "vip_id": 1
    }
  ],
  "name": vsvip_name,
}

#Send POST information via POST
resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

<Response [201]>
- New vsvip named vsvip-new-001 CREATED


We can extract some information from the response received from the AVI Controller. In this example, we can extract the allocated IP address from the internal IPAM. 

In [29]:
print("The allocated IP Address for the VSVIP object is: " + json.loads(resp.text)["vip"][0]["ip_address"]["addr"])

The allocated IP Address for the VSVIP object is: 192.168.1.53


### Example 2.2: Extracting references using api call instead of full values

A smarter way to populate some of the fields is by calling recurrently the API to get the value. As an example below, instead of providing the full network_ref value, we are just querying to the API where the name correspond to the intended network name _home-network_ with is much more simple way

In [32]:
# Define POST parameters
url_path="vsvip"
vsvip_name = "vsvip-new-002"
network_name = "home-network"


body = {
  "vip": [
    {
      "enabled": "true",
      "auto_allocate_ip": "true",
      "auto_allocate_ip_type": "V4_ONLY",
      "ipam_network_subnet": {
          "network_ref": '/api/network?name='+network_name,
          "subnet": {
            "ip_addr": {
               "addr": "192.168.1.0",
               "type": "V4"
            },
            "mask": 24
        }
      },
      "vip_id": 1
    }
  ],
  "name": vsvip_name,
}

#Send BODY information via POST
resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

#Shows allocated IP address.
print()
print("The allocated IP Address for the VSVIP object is: " + json.loads(resp.text)["vip"][0]["ip_address"]["addr"])

<Response [201]>
- New vsvip named vsvip-new-002 CREATED

The allocated IP Address for the VSVIP object is: 192.168.1.54


### Example 2.3.- Creating new object using existing one as a template

As you can tell, sometimes it could be really hard to craft the body from scratch since you need to be sure of all the mandatory fields and sintax. A better approach might be to extract (read) the information of an existing object and use this as a template. This allows you to clone JSON body and change the required information to create a new object without worrying that much about sintax and name of fields. 

Following example will use a SOURCE serviceenginegroup as template and after changing unique keys such as the name it will create a new cloned object. 

In [38]:
# Cloning an object from an exising one after changing required parameters

# Define GET parameters. Type SOURCE service engine group 
url_path="serviceenginegroup"
src_name = "MAD-SEG001"
query = {
   "name": src_name 
}

resp = api.get(url_path, params=query)
print ("Sending request to following URL: " +resp.url)

if resp.status_code in range(200, 299):
    print(resp)
    print(resp.reason)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Extracting results contents
if (json.loads(resp.text)['count']==0):
    print()
    print("Empty response for name: " +src_name)
else:
    print("Data found for name: " +src_name)
    data = json.loads(resp.text)["results"][0]


Sending request to following URL: https://192.168.1.15/api/serviceenginegroup?name=MAD-SEG001
<Response [200]>
OK
Data found for name: MAD-SEG001


Now we have the response we can create a new object with just the obtained response and updating the required fields with new values as shown below:

In [47]:
# Define new parameters
new_fields = {
   "name": "NEW-SEGROUP-003",
   "max_vs_per_se": 22
}

# Update dictionary overriding new fields values 
data.update(new_fields)

# Print modified items
print("Updated fields are: ")
for item in new_fields: 
    print("- "+item+": " +str(data[item]))

Updated fields are: 
- name: NEW-SEGROUP-003
- max_vs_per_se: 22


Now we have the new body, we just need to invoke the POST method again pushing the new body as part of the request. 

In [51]:
#Send BODY information via POST
url_path = "serviceenginegroup"
body = data
resp = api.post (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+body['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)

<Response [201]>
- New serviceenginegroup named NEW-SEGROUP-003 CREATED


## 3.- Modify (PUT) Method

PUT is the method used to make modifications of an existing object. As in the POST method, a body containing the information we want to update is required. Another important point is the uuid since the path of the url generally is comprised of the url + the uuid of the target object.  

In this example we will change the name of an existing virtual service. We need to get the uuid before to proceed with name change. Also, override required fields (name in this case) with new values. 

In [66]:
# Get the Virtual Service configuration of a given name to extract uuid
vs_name = "vs-example-org1"
vs = api.get_object_by_name("virtualservice", vs_name)

# Extract uuid from response
vs_uuid = vs["uuid"]

# Override name field with new name
vs["name"] = "vs-example-org1-new"

In [68]:
# Define PUT parameters
url_path="virtualservice/"+vs_uuid
body = vs 

#Send BODY information via PUT
resp = api.put (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
    print()
else:
    print('Error in modifying '+url_path+' :%s' % resp.text)

<Response [200]>
- Object virtualservice/virtualservice-6bd64913-d5f4-41d7-ade9-eb2e7c003b23 named vs-example-org1 modified OK



## 4.- Delete (DELETE) Method

Last method is the delete method that, as you can guess is used for deleting an existing object. 

In [76]:
# Get the Virtual Service configuration of a given name to extract uuid
seg_name = "NEW-SEGROUP-003"
seg = api.get_object_by_name("serviceenginegroup", seg_name)

# Extract uuid from response
seg_uuid = seg["uuid"]

In [77]:
seg_uuid

'serviceenginegroup-62be084f-212f-4f67-a169-43d2869ed62f'

In [78]:
# Define DELETE parameters
url_path="serviceenginegroup/"+seg_uuid
body = seg

#Send BODY information via DELETE
resp = api.delete(url_path)

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " deleted", resp.reason)#, resp.text)
    print()
else:
    print('Error in deleting '+url_path+' :%s' % resp.text)

<Response [204]>
- Object serviceenginegroup/serviceenginegroup-62be084f-212f-4f67-a169-43d2869ed62f named NEW-SEGROUP-003 deleted NO CONTENT

